<a href="https://colab.research.google.com/github/mheamahfoud/competitions/blob/main/Visual_Pollution_Detection_Using_Yolo_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Visual Pollution Detection


This solution  is based on the [YOLOv7 repository](https://github.com/WongKinYiu/yolov7) by WongKinYiu. 




### **Steps Covered in this solution**

To train our detector we take the following steps:

* Install YOLOv7 dependencies
* load dataset and anallysis
* split dataset for training
* Evaluate YOLOv7 performance




### Preparing a  Dataset

In this notBook, we download dataset as zip and extract 

In [ ]:
!nvidia-smi

#Global Variable

In [ ]:
#If you want to train model please change value of variable to True
IS_Training=False

##Load Libraries

In [ ]:
import os
import math
import numpy as np
import pandas as pd
import torch
from torch import nn
import cv2
import PIL
import matplotlib as mpl
from matplotlib import pyplot as plt
import glob
import os ,shutil
pd.set_option('display.max_columns', 10)
from tqdm.auto import tqdm
import argparse
from pathlib import Path
from pandas.core.algorithms import unique
from os import path        

#Load dataset 

In [ ]:
%cd /content

In [ ]:
dataset_zip_path = 'path_to_dataset_zip'
!unzip -q $dataset_zip_path -d /content/
!rm $dataset_zip_path
dataset_path='/content/dataset'

#Utils

In [ ]:
#function for drawing box on a image
def draw_boundbox(img, box):
    x0,y0,x1,y1 = box[0], box[1], box[2], box[3]
    color = (255, 0, 0)
    img = cv2.rectangle(img, (x0, y0), (x1, y1), color,10)
    return img

def draw_boundboxes(img, boxes):
    for box in boxes:
        img = draw_boundbox(img, box)
    return img

##convert form [xmin ymin xmax ymax] to [x_centered, y_centered, width, height] normalized
def convert_coordinates(size, box, scale):
    assert scale > 0.86, "scale is too small"
    dw = 1.0/size[0]
    dh = 1.0/size[1]
    x = (box[0]+box[1])/2.0
    y = (box[2]+box[3])/2.0
    w = scale * (box[1]-box[0])
    h = scale * (box[3]-box[2])
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

#function to create.txt yolo format
def csv_to_txt(data_set, out_path, class_labels, scale=1.0):
    # Read Csv 
    df = data_set
     # Group rows based on image_name because single file may have multiple annotations
    for name, group in df.groupby('image_path'):
        # Create filename
        fname_out = os.path.join(out_path, name.split(".")[0] + '.txt')
        # Open txt file to write
        with open(fname_out, "w") as f:
            # Iter through each bbox
            for row_index, row in group.iterrows():
                xmin = row['xmin']
                ymin = row['ymin']
                xmax = row['xmax']
                ymax = row['ymax']
                width = 1920
                height = 1080
                # Get label index
                label_str = str(math.trunc(class_labels[row['name']]))
                b = (float(xmin), float(xmax), float(ymin), float(ymax))
                # Convert bbox from pascal voc format to yolo txt format
                bb = convert_coordinates((width,height), b, scale)
                # Write into file
                f.write(label_str + " " + " ".join([("%.6f" % a) for a in bb]) + '\n')


 #function to create dataset(train,valid)               
def create_dataset(data,path_images,path_labels,dataset_path,class_labels=None):
    #craete labels
    if class_labels is not None:
      csv_to_txt(data ,path_labels,class_labels)
    #create images
   # Path(path_images).mkdir(parents=True, exist_ok=True)
    for image_path in list(data['image_path']):
        src = os.path.join(f'{dataset_path}/images/{image_path}')
        dst = os.path.join(f'{path_images}/{image_path}')
       # print(path.exists(dst))
        if not path.exists(dst) :
               #print(path.exists(dst))
               shutil.copy(src, dst)
        
        
        
def generate_images_path(dataset_path,data_type):
    fname_out = os.path.join( custom_data_path, data_type + '.txt')
    with open(fname_out, "w") as f:
        for path in os.listdir(dataset_path):
           # check if current path is a file

           if os.path.isfile(os.path.join(dataset_path, path)):
                f.write( os.path.join(dataset_path, path) + '\n')


font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 12}

plt.rc('font', **font)
#function tp draw images with boxs
def plotImagesAndLabels(dataset_path,dataset_filenames,dataset_label_box):
    fig = plt.figure(figsize=(16, 16))
    columns =3
    rows = 3
    index=1
    
    for k in range(9):
        img_name = np.random.choice(list(dataset_filenames))
        #print(img_name)
        for item in dataset_label_box[img_name]:
                test_img = cv2.cvtColor(cv2.imread( f'{dataset_path}/images/{img_name}', -1), cv2.COLOR_BGR2RGB)    
                test_img_bb = draw_boundbox(test_img, item[2])
                if index < 10 :
                    fig.add_subplot(rows, columns, index)
                    plt.imshow(test_img_bb)
                    plt.title(item[1])
                    plt.text(100, 100, img_name,)
                    index+=1
                break
               # fig.add_subplot(rows, columns, k)
                #plt.figure(figsize=(10, 10))
            

    
    plt.show()

#Analysis Dataset

In [ ]:
#original classes ordered by  depend on  train.csv
orginal_classes = ['GRAFFITI',
    'FADED_SIGNAGE',
    'POTHOLES',                                                                                                                                                    
    'GARBAGE',
    'CONSTRUCTION_ROAD',
    'BROKEN_SIGNAGE',
    'BAD_STREETLIGHT',
    'BAD_BILLBOARD',
    'SAND_ON_ROAD',
    'CLUTTER_SIDEWALK',
    'UNKEPT_FACADE']




In [ ]:
#dictiony class  map to  label (orginal dataset)
orginal_class_map_label_dictionery = {idx: clc for idx, clc in enumerate(orginal_classes)}
orginal_class_map_label_dictionery

In [ ]:
#dictiony label   map to  class  (orginal dataset)
orginal_label_map_class_dictionery = {clc: idx for idx, clc in enumerate(orginal_classes)}
orginal_label_map_class_dictionery



In [ ]:
#define df_dataset
df_origin_dataset=pd.read_csv(f'{dataset_path}/train.csv')
df_dataset = df_origin_dataset.copy()
df_test=pd.read_csv(f"{dataset_path}/test.csv")
dataset_filenames = df_dataset['image_path'].to_list()

##Check width and height o dataset

In [ ]:
from PIL import Image
width_ar=[];
height_ar=[];
for image_name in list(dataset_filenames):
    img = Image.open(f'{dataset_path}/images/{image_name}')
    width, height = img.size
    width_ar.append(width)
    height_ar.append(height)
res_width = all(ele == 1920 for ele in width_ar)
res_height = all(ele == 1080 for ele in height_ar)
print(res_width)
print(res_height)
print(len(height_ar))

##Fix bounding boxes

In [ ]:
#fix bounding box(multiply by 2, check negative values , values bigger than  size image)
max_vals = [1920, 1920, 1080, 1080]
for idx, col in enumerate(['xmin', 'xmax', 'ymin', 'ymax']):
    df_dataset[col] = df_dataset[col].astype(float)
    df_dataset[col] = df_dataset[col]*2
    df_dataset.loc[df_dataset[col] < 0, col] = 0
    df_dataset.loc[df_dataset[col] > max_vals[idx]-2, col ]= max_vals[idx]-2
    df_dataset[col] = df_dataset[col].astype(int)


df_dataset.head()

In [ ]:
#print min max values
for col in  ['xmin', 'xmax', 'ymin', 'ymax']:
  print(f'{col}: {df_dataset[col].min()} {df_dataset[col].max()}')

##Remove rare classes

In [ ]:
df_dataset['name'].value_counts().plot(kind='bar')

In [ ]:
#remove BAD_STREETLIGHT class as there is only one item
df_dataset = df_dataset.groupby('name').filter(lambda x: len(x)  >1)

#define class after removed
df_dataset['name'].value_counts()
classes_after_remove = df_dataset.name.unique().tolist()
classes_after_remove



In [ ]:
class_map_label_after_remove = {idx: clc for idx, clc in enumerate(classes_after_remove)}
class_map_label_after_remove

In [ ]:
label_map_class_after_remove = {clc: idx for idx, clc in enumerate(classes_after_remove)}
label_map_class_after_remove

##Plot images with labels and bounding box

In [ ]:
dataset_label_box = {}
for row in tqdm(df_dataset.iterrows(),total=len(df_dataset)):
    info = row[1]
    xmax, xmin, ymax, ymin = info['xmax'], info['xmin'], info['ymax'], info['ymin']
    image_path = info['image_path']
    
    dataset_label_box[image_path] = dataset_label_box.get(image_path,[])
    dataset_label_box[image_path].append((info['class'], info['name'], [xmin, ymin, xmax, ymax]))
dataset_label_box;

In [ ]:
plotImagesAndLabels(dataset_path,dataset_filenames,dataset_label_box);

##Split images to valid and train

In [ ]:
from sklearn.model_selection import train_test_split
images_pathes = df_dataset.image_path.unique()
train_image_pathes, valid_image_pathes = train_test_split(images_pathes, test_size=0.10, random_state=42)


In [ ]:
#Check classes distribution in train and valid
for cls in label_map_class_after_remove.keys():
  cls_train = len(df_dataset[(df_dataset.image_path.isin(train_image_pathes)) & (df_dataset.name == cls)])
  cls_valid = len(df_dataset[(df_dataset.image_path.isin(valid_image_pathes)) & (df_dataset.name == cls)])
  print(f"{cls}: train: {cls_train} valid {cls_valid}")

In [ ]:
#create folder for train , test and  valid dataset deponding  on yolo format
if os.path.isdir('/content/custom_data/'):
    shutil.rmtree('/content/custom_data/')
custom_data_path='/content/custom_data/';
os.makedirs(custom_data_path, exist_ok=True)

labels_path_test=custom_data_path + 'test/labels/';
images_path_test=custom_data_path + 'test/images/';
os.makedirs(labels_path_test, exist_ok=True)
os.makedirs(images_path_test, exist_ok=True)

labels_path_train=custom_data_path + 'train/labels/';
images_path_train=custom_data_path + 'train/images/';
os.makedirs(labels_path_train, exist_ok=True)
os.makedirs(images_path_train, exist_ok=True)

labels_path_valid=custom_data_path + 'valid/labels/';
images_path_valid=custom_data_path+ 'valid/images/';
os.makedirs(labels_path_valid, exist_ok=True)
os.makedirs(images_path_valid, exist_ok=True)


In [ ]:
# copy valid images
df_valid = df_dataset[(df_dataset.image_path.isin(valid_image_pathes))]
create_dataset(df_valid,images_path_valid,labels_path_valid,dataset_path,label_map_class_after_remove)

In [ ]:
# copy train  images
df_train = df_dataset[(df_dataset.image_path.isin(train_image_pathes))]
create_dataset(df_train,images_path_train,labels_path_train,dataset_path,label_map_class_after_remove)

In [ ]:
# copy test  images
create_dataset(df_test,images_path_test,labels_path_test,dataset_path)
_, _, files = next(os.walk('/content/custom_data/test/images'))
file_count = len(files)
file_count

#Install Dependency Yolo



In [ ]:
!git clone https://github.com/SkalskiP/yolov7.git
%cd yolov7
!git checkout fix/problems_associated_with_the_latest_versions_of_pytorch_and_numpy
!pip install -r requirements.txt

#Train Models

In [ ]:
%cd /content/yolov7

In [ ]:
# download COCO starting checkpoint
%cd /content/yolov7
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7_training.pt

In [ ]:
#yaml file depending on yolo format
import os
import yaml

config = {
   'train':'/content/custom_data/train/images',
    'val': '/content/custom_data/valid/images' ,
    'nc': len(classes_after_remove),
    'names':  classes_after_remove
}
 
with open(os.path.join("data/", "custom_data.yaml"), "w") as file:
    yaml.dump(config, file, default_flow_style=False)

##change lr and lf

In [ ]:
%%writefile /content/yolov7/data/hyp.scratch.custom.yaml

lr0: 0.001  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.01  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.3  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 0.7  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.4  # image HSV-Value augmentation (fraction)
degrees: 0.0  # image rotation (+/- deg)
translate: 0.2  # image translation (+/- fraction)
scale: 0.5  # image scale (+/- gain)
shear: 0.0  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: 0.0  # image flip up-down (probability)
fliplr: 0.5  # image flip left-right (probability)
mosaic: 1.0  # image mosaic (probability)
mixup: 0.0  # image mixup (probability)
copy_paste: 0.0  # image copy paste (probability)
paste_in: 0.0  # image copy paste (probability), use 0 for faster training
loss_ota: 1 # use ComputeLossOTA, use 0 for faster training

In [ ]:
#disable wandb else enable if you want to monitor training
!pip install wandb
!wandb disabled

In [ ]:
# #If you want to train model please change value of variable(IS_Training) to True and run this cellto begin training
if IS_Training :
    !python train.py  --batch-size 8 \
                    --epoch 30 \
                    --data /content/yolov7/data/custom_data.yaml \
                    --img 640 640 \
                    --cfg cfg/training/yolov7.yaml \
                    --weights 'yolov7_training.pt' \
                    --name yolov7 \
                    --hyp data/hyp.scratch.custom.yaml \
                    --save_period 1 \
                      --workers 8      \
                      --project /content/gdrive/MyDrive/Yolox7 \
                    --freeze  1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30

#Evaluation

###evaluate.py file

In [ ]:
%%writefile detect.py
import argparse
import time
from pathlib import Path
import pandas as pd
import cv2
import torch
import torch.backends.cudnn as cudnn
from numpy import random

from models.experimental import attempt_load
from utils.datasets import LoadStreams, LoadImages
from utils.general import check_img_size, check_requirements, check_imshow, non_max_suppression, apply_classifier, \
    scale_coords, xyxy2xywh, strip_optimizer, set_logging, increment_path
from utils.plots import plot_one_box
from utils.torch_utils import select_device, load_classifier, time_synchronized, TracedModel


def detect(save_img=False):
    source, weights, view_img, save_txt, imgsz, trace = opt.source, opt.weights, opt.view_img, opt.save_txt, opt.img_size, not opt.no_trace
    save_img = not opt.nosave and not source.endswith('.txt')  # save inference images
    webcam = source.isnumeric() or source.endswith('.txt') or source.lower().startswith(
        ('rtsp://', 'rtmp://', 'http://', 'https://'))

    # Directories
    save_dir = Path(increment_path(Path(opt.project) / opt.name, exist_ok=opt.exist_ok))  # increment run
    (save_dir / 'labels' if save_txt else save_dir).mkdir(parents=True, exist_ok=True)  # make dir
    #print(f"  save_dir is saved in: {save_dir}")
    # Initialize
    set_logging()
    device = select_device(opt.device)
    half = device.type != 'cpu'  # half precision only supported on CUDA

    # Load model
    model = attempt_load(weights, map_location=device)  # load FP32 model
    stride = int(model.stride.max())  # model stride
    imgsz = check_img_size(imgsz, s=stride)  # check img_size

    if trace:
        model = TracedModel(model, device, opt.img_size)

    if half:
        model.half()  # to FP16

    # Second-stage classifier
    classify = False
    if classify:
        modelc = load_classifier(name='resnet101', n=2)  # initialize
        modelc.load_state_dict(torch.load('weights/resnet101.pt', map_location=device)['model']).to(device).eval()

    # Set Dataloader
    vid_path, vid_writer = None, None
    if webcam:
        view_img = check_imshow()
        cudnn.benchmark = True  # set True to speed up constant image size inference
        dataset = LoadStreams(source, img_size=imgsz, stride=stride)
    else:
        dataset = LoadImages(source, img_size=imgsz, stride=stride)

    # Get names and colors
    names = model.module.names if hasattr(model, 'module') else model.names
    colors = [[random.randint(0, 255) for _ in range(3)] for _ in names]
#   submite file
    orginal_classes = ['GRAFFITI',
            'FADED_SIGNAGE',
            'POTHOLES',                                                                                                                                                    
            'GARBAGE',
            'CONSTRUCTION_ROAD',
            'BROKEN_SIGNAGE',
            'BAD_STREETLIGHT',
            'BAD_BILLBOARD',
            'SAND_ON_ROAD',
            'CLUTTER_SIDEWALK',
            'UNKEPT_FACADE'
        ]


    class_orginal_class = {clc: idx for idx, clc in enumerate(orginal_classes)}
    df_submission =pd.DataFrame([], columns=['class', 'image_path','name','xmax','xmin','ymax','ymin'])
    # Run inference
    if device.type != 'cpu':
        model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))  # run once
    old_img_w = old_img_h = imgsz
    old_img_b = 1

    t0 = time.time()
    for path, img, im0s, vid_cap in dataset:
        #print(path.split('/')[-1])
        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        # Warmup
        if device.type != 'cpu' and (old_img_b != img.shape[0] or old_img_h != img.shape[2] or old_img_w != img.shape[3]):
            old_img_b = img.shape[0]
            old_img_h = img.shape[2]
            old_img_w = img.shape[3]
            for i in range(3):
                model(img, augment=opt.augment)[0]

        # Inference
        t1 = time_synchronized()
        with torch.no_grad():   # Calculating gradients would cause a GPU memory leak
            pred = model(img, augment=opt.augment)[0]
        t2 = time_synchronized()

        # Apply NMS
        pred = non_max_suppression(pred, opt.conf_thres, opt.iou_thres, classes=opt.classes, agnostic=opt.agnostic_nms)
        t3 = time_synchronized()

        # Apply Classifier
        if classify:
            pred = apply_classifier(pred, modelc, img, im0s)

        # Process detections
        for i, det in enumerate(pred):  # detections per image
            if webcam:  # batch_size >= 1
                p, s, im0, frame = path[i], '%g: ' % i, im0s[i].copy(), dataset.count
            else:
                p, s, im0, frame = path, '', im0s, getattr(dataset, 'frame', 0)
           # print(f'p.name {p.name}')
            p = Path(p)  # to Path
            save_path = str(save_dir / p.name)  # img.jpg
            txt_path = str(save_dir / 'labels' / p.stem) + ('' if dataset.mode == 'image' else f'_{frame}')  # img.txt
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
            if len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

                # Print results
                for c in det[:, -1].unique():
                    n = (det[:, -1] == c).sum()  # detections per class
                    s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string

                # Write results
                confice=0;
                index=0
                res=[]
                #print(reversed(det))
                for k in reversed(det):
              
                        *xyxy, conf, cls = k
                        # label = f'{names[int(cls)]} {conf:.2f}'
                        x=xyxy
                        c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
                        df_submission=df_submission.append({'class':int(class_orginal_class[names[int(cls)]]), 'image_path':path.split('/')[-1], 'name':names[int(cls)],'xmax':int(x[2]/2),'xmin':int(x[0]/2),'ymax':int(x[3]/2),'ymin':int(x[1]/2) }, ignore_index=True)


            else :
                        df_submission=df_submission.append({'class':'', 'image_path':path.split('/')[-1], 'name':'','xmax':'','xmin':'','ymax':'','ymin':'' }, ignore_index=True)

                 # Stream results
            if view_img:
                cv2.imshow(str(p), im0)
                cv2.waitKey(1)  # 1 millisecond

            # Save results (image with detections)
           

    if save_txt or save_img:
        s = f"\n{len(list(save_dir.glob('labels/*.txt')))} labels saved to {save_dir / 'labels'}" if save_txt else ''
        print(f"Results saved to {save_dir}{s}")

    #print(f'Done. ({time.time() - t0:.3f}s)')
    df_submission.to_csv('/content/submission.csv',index=False)

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--weights', nargs='+', type=str, default='yolov7.pt', help='model.pt path(s)')
    parser.add_argument('--source', type=str, default='inference/images', help='source')  # file/folder, 0 for webcam
    parser.add_argument('--img-size', type=int, default=640, help='inference size (pixels)')
    parser.add_argument('--conf-thres', type=float, default=0.25, help='object confidence threshold')
    parser.add_argument('--iou-thres', type=float, default=0.45, help='IOU threshold for NMS')
    parser.add_argument('--device', default='', help='cuda device, i.e. 0 or 0,1,2,3 or cpu')
    parser.add_argument('--view-img', action='store_true', help='display results')
    parser.add_argument('--save-txt', action='store_true', help='save results to *.txt')
    parser.add_argument('--save-conf', action='store_true', help='save confidences in --save-txt labels')
    parser.add_argument('--nosave', action='store_true', help='do not save images/videos')
    parser.add_argument('--classes', nargs='+', type=int, help='filter by class: --class 0, or --class 0 2 3')
    parser.add_argument('--agnostic-nms', action='store_true', help='class-agnostic NMS')
    parser.add_argument('--augment', action='store_true', help='augmented inference')
    parser.add_argument('--update', action='store_true', help='update all models')
    parser.add_argument('--project', default='runs/detect', help='save results to project/name')
    parser.add_argument('--name', default='exp', help='save results to project/name')
    parser.add_argument('--exist-ok', action='store_true', help='existing project/name ok, do not increment')
    parser.add_argument('--no-trace', action='store_true', help='don`t trace model')
    opt = parser.parse_args()
    print(opt)
    #check_requirements(exclude=('pycocotools', 'thop'))

    with torch.no_grad():
        if opt.update:  # update all models (to fix SourceChangeWarning)
            for opt.weights in ['yolov7.pt']:
                detect()
                strip_optimizer(opt.weights)
        else:
            detect()

###detect

In [ ]:
#Download pretrained model, https://medium.com/@acpanjan/download-google-drive-files-using-wget-3c2c025a8b99
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=15gZladqILTKQCZr1S2oLdWp1970T7kCy' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=15gZladqILTKQCZr1S2oLdWp1970T7kCy" -O best.pt && rm -rf /tmp/cookies.txt

In [ ]:
15gZladqILTKQCZr1S2oLdWp1970T7kCy
15gZladqILTKQCZr1S2oLdWp1970T7kCy
15gZladqILTKQCZr1S2oLdWp1970T7kCy
https://drive.google.com/file/d/15gZladqILTKQCZr1S2oLdWp1970T7kCy/view?usp=share_link

In [ ]:
!python detect.py --weights 'best.pt' --img-size 640 --source /content/custom_data/test/images  --conf 0.3

In [ ]:
# after run this cell,  submission file will be downloaded
import numpy as np
def expand_bb(row, ratio=1.05):
  #Expand bounding box by ratio
  xmin, xmax, ymin, ymax = row['xmin'], row['xmax'], row['ymin'], row['ymax']
  if np.isnan(xmin):
    return row
  xc = (xmin+xmax)//2
  yc = (ymin+ymax)//2
  xmin = xc - ratio * (xmax-xmin) //2
  xmax = xc + ratio * (xmax-xmin) //2
  ymin = yc - ratio * (ymax-ymin) //2
  ymax =  yc + ratio * (ymax-ymin) //2
  row['xmin'] = max(int(xmin), 0)
  row['xmax'] = min(int(xmax), 1920//2)
  row['ymin'] = max(int(ymin), 0)
  row['ymax'] = min(int(ymax), 1920//2)
  return row

df = pd.read_csv("/content/submission.csv")
df2 = df.copy()
df2 = df2.apply(expand_bb, axis=1)
df2.to_csv("/content/submission_expand_bb.csv", index=False)
from google.colab import files
files.download('/content/submission_expand_bb.csv')